# function stores

In [1]:
def _classification_report(prediction , nameOfModel) : 
    #PREPARE SECCTION
    target_names = df_label["sleep"].unique().sort()
    clf_rbf_raw = classification_report(y_test,prediction ,target_names=target_names  ,output_dict=True)
    clf_rbf = pd.DataFrame(clf_rbf_raw)
    
    #PLOTING SECCTION
    fig = px.imshow(clf_rbf.iloc[:-1, :].T, text_auto=True ,aspect="auto",color_continuous_scale='GnBu'  )
    fig.update_layout(height=400, width=500, title_text='Classification Report of '+str(nameOfModel) +"SVM")
    fig.show()

# import lib

In [266]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from sklearn.model_selection import  GridSearchCV ,RandomizedSearchCV
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.metrics import plot_confusion_matrix
import plotly.graph_objects as go
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.svm import SVC
import warnings
warnings.filterwarnings('ignore')
from scipy import stats

# (a) Load and Prepare Data

In [3]:
# Load data from csv 3 files
# acceleration.txt, heartrate.txt, labeled_sleep.txt
acceleration = "./Resources/46343_acceleration.txt"
heartrate = "./Resources/46343_heartrate.txt"
labeled_sleep="./Resources/46343_labeled_sleep.txt"
init_acc = pd.read_csv(acceleration, sep = ' ',names=['timedelta', 'accX', 'accY', 'accZ'])
init_heartrate = pd.read_csv(heartrate, sep = ',',names=['timedelta', 'heartrate'])
init_sleep = pd.read_csv(labeled_sleep, sep = ' ',names=['timedelta', 'sleep'])

acceleration2 = "./Resources/844359_acceleration.txt"
heartrate2 = "./Resources/844359_heartrate.txt"
labeled_sleep2="./Resources/844359_labeled_sleep.txt"
init_acc2 = pd.read_csv(acceleration2, sep = ' ',names=['timedelta', 'accX', 'accY', 'accZ'])
init_heartrate2 = pd.read_csv(heartrate2, sep = ',',names=['timedelta', 'heartrate'])
init_sleep2 = pd.read_csv(labeled_sleep2, sep = ' ',names=['timedelta', 'sleep'])

In [5]:
init_acc2.head()

,timedelta,accX,accY,accZ
0,-500.492604,-0.023804,-0.535217,-0.832169
1,-500.474720,-0.015472,-0.535690,-0.833450
2,-500.455065,-0.019379,-0.528381,-0.834518
3,-500.434675,-0.013473,-0.531311,-0.836334
4,-500.414981,-0.015366,-0.533752,-0.839294


In [6]:
init_heartrate2.head()

,timedelta,heartrate
0,-604720.29398,68
1,-604438.91905,64
2,-604032.54366,63
3,-603757.54405,64
4,-603454.66864,64


In [7]:
init_sleep2.head()

,timedelta,sleep
0,0,-1
1,30,-1
2,60,-1
3,90,-1
4,120,-1


In [8]:
acc_max_date = init_acc2["timedelta"].max()
acc_min_date = init_acc2["timedelta"].min()
print("acceleration start :" ,acc_min_date)
print("acceleration end :" ,acc_max_date)

acceleration start : -500.492604017
acceleration end : 28828.1484408


In [9]:
sleep_min = init_sleep2["timedelta"].min()
sleep_max = init_sleep2["timedelta"].max()
print("sleep label start :" ,sleep_min)
print("sleep label end :" ,sleep_max)

sleep label start : 0
sleep label end : 28440


In [10]:
hrate_min = init_heartrate2["timedelta"].min()
hrate_max = init_heartrate2["timedelta"].max()
print("heart rate start :" ,hrate_min)
print("heart rate end :" ,hrate_max)

heart rate start : -604720.29398
heart rate end : 28993.7883699


### select only intersected timedelta (ACC, HeartR, SleepL) (ช่วงเวลาที่มีข้อมูลใกล้กัน)

In [371]:
seleted_min  = 100
seleted_max = 10000

p2min = 7000
p2max = 12000

In [372]:
acc_new = init_acc[(init_acc['timedelta'] > seleted_min ) & (init_acc['timedelta'] <= seleted_max)]
heartRate_new = init_heartrate[(init_heartrate['timedelta'] >= seleted_min ) & (init_heartrate['timedelta'] < seleted_max)]
sleepLabel_new = init_sleep[(init_sleep['timedelta'] >= seleted_min ) &(init_sleep['timedelta'] <= seleted_max)]

In [373]:
p2_acc = init_acc[(init_acc2['timedelta'] > p2min ) & (init_acc2['timedelta'] <= p2max)]
p2_hr = init_heartrate[(init_heartrate2['timedelta'] >= p2min ) & (init_heartrate2['timedelta'] < p2max)]
p2_sleep = init_sleep[(init_sleep2['timedelta'] >= p2min ) &(init_sleep2['timedelta'] <= p2max)]

# Convert to datetime and round to second,

In [374]:
acc_new["timedelta"] = pd.DataFrame(pd.to_timedelta(acc_new['timedelta'], unit='s').round('s'))
p2_acc["timedelta"] = pd.DataFrame(pd.to_timedelta(p2_acc['timedelta'], unit='s').round('s'))

# Average rounding duplicated time

In [375]:
df_acc_X = acc_new.groupby('timedelta')['accX'].mean().reset_index()
df_acc_Y = acc_new.groupby('timedelta')['accY'].mean().reset_index()
df_acc_Z = acc_new.groupby('timedelta')['accZ'].mean().reset_index()

p2x = p2_acc.groupby('timedelta')['accX'].mean().reset_index()
p2y = p2_acc.groupby('timedelta')['accY'].mean().reset_index()
p2z = p2_acc.groupby('timedelta')['accZ'].mean().reset_index()

In [376]:
acc_df = df_acc_X.set_index('timedelta').join(df_acc_Y.set_index('timedelta')).join(df_acc_Z.set_index('timedelta'))
p2_acc_df = p2x.set_index('timedelta').join(p2y.set_index('timedelta')).join(p2z.set_index('timedelta'))

# (c) Load and Prepare Data (Heart rate)

In [377]:
heartRate_new["timedelta"] = pd.DataFrame(pd.to_timedelta(heartRate_new['timedelta'], unit='s').round('1s'))
p2_hr["timedelta"] = pd.DataFrame(pd.to_timedelta(p2_hr['timedelta'], unit='s').round('1s'))

In [378]:
# Group duplicate time
heartRate_new = heartRate_new.groupby('timedelta')['heartrate'].mean().reset_index()
p2_hr = p2_hr.groupby('timedelta')['heartrate'].mean().reset_index()

In [379]:
resample_rule = '1s'
heartrate_df = heartRate_new.set_index('timedelta').resample(resample_rule,).median().ffill()
p2_hr_df = p2_hr.set_index('timedelta').resample(resample_rule,).median().ffill()

# (c) Load and Prepare Data (Sleep Label)

In [380]:
sleepLabel_new["timedelta"] = pd.DataFrame(pd.to_timedelta(sleepLabel_new['timedelta'], unit='s').round('1s'))
p2_sleep["timedelta"] = pd.DataFrame(pd.to_timedelta(p2_sleep['timedelta'], unit='s').round('1s'))


In [381]:
sleeplabel_df = sleepLabel_new.set_index('timedelta').resample(resample_rule,).median().ffill()
p2_sleep_df = p2_sleep.set_index('timedelta').resample(resample_rule,).median().ffill()
p2_sleep_df

,sleep
timedelta,
0 days 01:57:00,2.0
0 days 01:57:01,2.0
0 days 01:57:02,2.0
0 days 01:57:03,2.0
0 days 01:57:04,2.0
...,...
0 days 03:19:56,1.0
0 days 03:19:57,1.0
0 days 03:19:58,1.0


# ( e) Merge Data and Standardized data

In [382]:
df2 = []
df2= pd.merge_asof(p2_acc_df, p2_hr_df, on='timedelta')
df2 = pd.merge_asof(df2, p2_sleep_df, on='timedelta')
df2

,timedelta,accX,accY,accZ,heartrate,sleep
0,0 days 01:30:34,0.469599,-0.147249,-0.871480,NaN,NaN
1,0 days 01:30:35,0.475812,-0.145139,-0.868174,NaN,NaN
2,0 days 01:30:36,0.472598,-0.146138,-0.870019,NaN,NaN
3,0 days 01:30:37,0.469959,-0.147274,-0.870844,NaN,NaN
4,0 days 01:30:38,0.469910,-0.147196,-0.870884,NaN,NaN
...,...,...,...,...,...,...
4988,0 days 02:53:42,-0.155810,-0.987706,-0.010401,79.0,5.0
4989,0 days 02:53:43,-0.156098,-0.987573,-0.010050,79.0,5.0
4990,0 days 02:53:44,-0.156004,-0.987456,-0.011153,79.0,5.0
4991,0 days 02:53:45,-0.155873,-0.987352,-0.011609,79.0,5.0


In [383]:
df = []
df = pd.merge_asof(acc_df, heartrate_df, on='timedelta')
df = pd.merge_asof(df, sleeplabel_df, on='timedelta')

In [384]:
df

,timedelta,accX,accY,accZ,heartrate,sleep
0,0 days 00:01:40,-0.376241,0.925141,-0.008848,83.0,NaN
1,0 days 00:01:41,-0.377814,0.925636,0.003382,83.0,NaN
2,0 days 00:01:42,-0.379298,0.924588,0.003643,83.0,NaN
3,0 days 00:01:43,-0.379019,0.925039,0.000995,83.0,NaN
4,0 days 00:01:44,-0.380270,0.924537,-0.000302,83.0,NaN
...,...,...,...,...,...,...
9896,0 days 02:46:36,-0.380194,0.612306,-0.694738,76.0,1.0
9897,0 days 02:46:37,-0.387287,0.612106,-0.690805,76.0,1.0
9898,0 days 02:46:38,-0.385302,0.614457,-0.689404,76.0,1.0
9899,0 days 02:46:39,-0.381994,0.614781,-0.691998,76.0,1.0


In [385]:
df.isnull().sum()

timedelta     0
accX          0
accY          0
accZ          0
heartrate     0
sleep        20
dtype: int64

# fill na sleep data wth 0

In [386]:
df['sleep'] = df["sleep"].fillna(value=0)
df['heartrate'] = df["heartrate"].fillna(df['heartrate'].median())

df2['sleep'] = df2["sleep"].fillna(value=0)
df2['heartrate'] = df2["heartrate"].fillna(df2['heartrate'].median())

# Standardized data

In [387]:
# Standardized data
feature_columns = ['accX', 'accY', 'accZ', 'heartrate']
label_columns = ['sleep']
df_feature = df[feature_columns]
df_label = df[label_columns]


In [388]:
std = StandardScaler()

In [389]:
norm_value = std.fit_transform(df_feature)
norm_p2 = std.fit_transform(df2_feature)

In [390]:
norm_df = pd.DataFrame(norm_value ,columns=feature_columns)

# CREATE 3D INPUT

In [391]:
df_feature.shape

(9901, 4)

In [392]:
# ------------ 1D to 3D feature-------------------------------
 # set sliding window parameter
sliding_window= 100
stride_step = 5
n_feature = 4
df_feature3D =[]
target_ = []
for t in range( 0 , len(df_feature) -sliding_window , stride_step):
    feature = np.array(df_feature.iloc[t:t + sliding_window])
    feR = feature.reshape(sliding_window , n_feature , 1)
    df_feature3D.append(feR)
    
    LB_raw  = np.array(df_label.iloc[t : t+slidingW])
    mode_val,count = stats.mode(LB_raw ,axis=None )
    target_.append(mode_val[0])

In [393]:
count

array([90])

In [394]:
df_feature3D = np.array(df_feature3D)

In [395]:
df_feature3D.shape

(1961, 100, 4, 1)

In [396]:
target_ = np.array(target_)

In [397]:
target_

array([-1., -1., -1., ...,  2.,  2.,  2.])

# 9.1.2 Train Test Split

In [399]:
# 2D -------------->
x_train, x_test, y_train, y_test = train_test_split( df_feature, df_label)

# 3D -------------->
x3d_train, x3d_test, target_train, target_test = train_test_split( df_feature3D, target_)

# 9.2 NN Model Train and Test

In [400]:
from sklearn.neural_network import MLPClassifier

In [401]:
# ------------ NN Architecture parameter -------------------------------
Hidden_Layer_param = (30, 30, 30)
mlp = MLPClassifier(hidden_layer_sizes = Hidden_Layer_param)
mlp

MLPClassifier(hidden_layer_sizes=(30, 30, 30))

# Fitting NN Model

In [402]:
y_train.shape

(7425, 1)

In [403]:
mlp.fit(x_train,y_train)

MLPClassifier(hidden_layer_sizes=(30, 30, 30))

In [404]:
nn_prediction  = mlp.predict(x_test)

In [405]:
nn_prediction

array([3., 5., 3., ..., 3., 3., 2.])

In [406]:
def _classification_report(prediction , nameOfModel) : 
    #PREPARE SECCTION
    target_names = df_label["sleep"].unique().sort()
    clf_rbf_raw = classification_report(y_test,prediction ,target_names=target_names  ,output_dict=True)
    clf_rbf = pd.DataFrame(clf_rbf_raw)
    
    #PLOTING SECCTION
    fig = px.imshow(clf_rbf.iloc[:-1, :].T, text_auto=True ,aspect="auto",color_continuous_scale='GnBu'  )
    fig.update_layout(height=400, width=500, title_text='Classification Report of '+str(nameOfModel) +"SVM")
    fig.show()

In [411]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test , nn_prediction)

array([[ 68,   2,   0,   0,   0,   0],
       [  6, 249,   0,  44,   3,   5],
       [  0,  29,   0,  20,   0,   0],
       [  0,  26,   0, 412,  48,  49],
       [  0,   2,   0, 132, 936,   0],
       [  0,   0,   0,  20,   0, 425]], dtype=int64)

In [413]:
print(classification_report(y_test , nn_prediction))

              precision    recall  f1-score   support

        -1.0       0.92      0.97      0.94        70
         0.0       0.81      0.81      0.81       307
         1.0       0.00      0.00      0.00        49
         2.0       0.66      0.77      0.71       535
         3.0       0.95      0.87      0.91      1070
         5.0       0.89      0.96      0.92       445

    accuracy                           0.84      2476
   macro avg       0.70      0.73      0.72      2476
weighted avg       0.84      0.84      0.84      2476



In [418]:
nn_pred_df = pd.DataFrame(nn_prediction)
nn_pred_df

,0
0,3.0
1,5.0
2,3.0
3,2.0
4,5.0
...,...
2471,5.0
2472,5.0
2473,3.0
2474,3.0


In [428]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'tensorflow'

In [427]:
# ------------ CNN Architecture parameter -------------------------------
# Nlayer (CNN, dense), Nnode, Activation
CNN_L1 = 16
CNN_L2 = 64
CNN_L3 = 128
D_L1 = 512
D_out = 6
Activation = "Relu"
Ker_size = (3,3)
Pooling_size = (2,1)
Input_shape = (slidingW, n_feature, 1)